# 修了課題④　CIFAR-10

**データセット**：CIFAR-10データセット（Canadian Institute For Advanced Research）（以下、CIFAR-10）は、

- ラベル「0」：airplane（飛行機）
- ラベル「1」：automobible（自動車）
- ラベル「2」：bird（鳥）
- ラベル「3」：cat（猫）
- ラベル「4」：deer（鹿）
- ラベル「5」：dog（犬）
- ラベル「6」：frog（カエル）
- ラベル「7」：horse（馬）
- ラベル「8」：ship（船）
- ラベル「9」：truck（トラック）

という10種類の「物体カラー写真」（乗り物や動物など）の画像データセットである。

CIFAR-10は、主に画像認識を目的としたディープラーニング／機械学習の研究や初心者向けチュートリアルで使われている。CIFAR-10は上記の通り10クラス（種類）となっており手軽に扱えるが、より複雑な内容として100クラス版であるCIFAR-100も提供されている。

CIFAR-10データセット全体は、

- 5万枚の訓練データ用（画像とラベル）
- 1万枚のテストデータ用（画像とラベル）
- 合計6万枚

で構成される（※「ラベル」= 正解を示す教師データ）。また各画像のフォーマットは、

- 24bit RGBフルカラー画像：RGB（赤色／緑色／青色）3色の組み合わせで、それぞれ「0」〜「255」の256段階
- 幅32x高さ32ピクセル：1つ分のデータが基本的に(3,32,32)もしくは(32,32,3)（=計3072画素）という多次元配列の形状となっており、最初もしくは最後の次元にある3要素がRGB値

となっている（※「ピクセル」=画素のこと。RGB形式であるため、簡単に画像化できる）。

**合格基準**：正解率 85%以上です

##必要なライブラリーのインポートとGoogleDriveへの接続

In [ ]:
#GoogleDriveへの接続を行う
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#必要なライブラリーのインポート
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#ダウンロードに必要なライブラリーのインポート
import pickle
from PIL import Image
import os

#データのダウンロードと正規化

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3' -O train.pickle

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN' -O val.pickle

In [ ]:
def parse_pickle(rawdata, dataset_name):
    for i in range(10):
        dir = dataset_name + "/" + f"{i:02d}"
        if not os.path.exists(dir):
            os.makedirs(dir)
    m = len(rawdata["data"])
    for i in range(m):
        filename = f'{i}.png'
        label = rawdata["label"][i]
        data = rawdata["data"][i]
        data = data.reshape(3, 32, 32)
        data = np.swapaxes(data, 0, 2)
        data = np.swapaxes(data, 0, 1)
        with Image.fromarray(data) as img:
            img.save(f"{dataset_name}/{label:02d}/{filename}")

train = {'label':[], 'data':[]}
with open('train.pickle', "rb") as fp:
  train = pickle.load(fp, encoding="latin-1")
parse_pickle(train, "train")

with open('val.pickle', "rb") as fp:
  val = pickle.load(fp, encoding="latin-1")
parse_pickle(val, "val")

##データ拡張とデータセット作成

In [ ]:
# データ拡張とデータセットを作成してください。

# モデルの構築

In [ ]:
# モデルを作成し、テストデータに対して予測結果を出力して下さい。

# 提出形式

## テスト用データセットのダウンロード

In [ ]:
# テスト用のデータ（正解ラベルなし）のダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1-T-luRcFf14qV_rR66B3groh8imA-8lo' -O test_data.pickle

In [ ]:
# 提出形式見本
# 提出データの形、タイトル
import pandas as pd
y_pred = pd.DataFrame(予測結果の変数, columns=['number'])
# csv形式で提出して下さい。
y_pred.to_csv('y_pred.csv')
y_pred